# Factors Driving Austin Housing Prices
## Join Appraisals to Housing Data
Ricardo Dunia (duniarh)
<br/>
Kevin John Cherian Joseph (kc43529)
<br/>
Arnob Mallick (am93746)

In [4]:
import difflib
import pandas as pd
import multiprocessing as mp

housing_df = pd.read_csv('./data/raw_austin_housing_data_w_crime_density_poi.csv', low_memory=False)

In [5]:
# Custom get_close_matches that returns indices instead of values
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_matches(addr, no, choices):
    best_match = -1
    if addr == '' or no == '':
        return best_match

    filtered_choices = { idx: choices['addr_street'][idx] for idx in choices['addr_street'] if choices['addr_no'][idx] == no }
    matches = []
    try:
        matches = process.extract(addr, filtered_choices, limit=3)
    except:
        return best_match

    # validate the matches
    for match in matches:
        idx = match[2]
        score = match[1]
        if choices['addr_no'][idx] == no and score > 80:
            best_match = idx
            break

    return best_match

In [6]:
# build Address
housing_df['Address'] = housing_df[['streetLine', 'state', 'zip']].apply(lambda x: ' '.join(x.map(str)), axis=1)

# remove addresses that aren't properly filled out
housing_df['Address'] = housing_df['Address'].apply(lambda x: x if str(x.split(' ')[0].isdigit()) else '')
housing_df = housing_df[housing_df['Address'] != '']

# extract house number and tirm the address
housing_df['Number'] = housing_df['Address'].apply(lambda x: x.split(' ')[0])
housing_df['Address'] = housing_df['Address'].apply(lambda x: ' '.join(x.split(' ')[1:-1]).replace('TX', '').strip())

# create an index column
housing_df['idx'] = housing_df.index

In [7]:
# store zip code data in bins
zip_bins = { }

# Function to take a house row and find it's appraisal match
def findAppraisal(housing_df_row):
    zip_str = str(int(housing_df_row[1]['zip']))
    if zip_str in zip_bins:
        match = find_matches(housing_df_row[1]['Address'], housing_df_row[1]['Number'], zip_bins[zip_str])
        if match != -1:
            idx = housing_df_row[1]['idx']
            return { 'idx': idx, 'val': zip_bins[zip_str]['appraisal'][match], 'match': True }
    return { 'match': False }

In [11]:
# function to add appraisal data
def add_appraisal_data(year):
    print('\n\nBuilding appraisal data for ' + year)
    global zip_bins

    # load data
    appraisal_df = pd.read_csv('./data/austin_appraisal_data_' + year + '.csv', low_memory=False)

    # Remove properties that are doing some business
    # appraisal_df = appraisal_df[appraisal_df['Doing Business As'].isnull()]

    # Eliminate rows from appraisal dataset with addresses that cannot be mapped
    appraisal_df = appraisal_df[appraisal_df['Property Address'].str.strip().str[0].str.isdigit()]

    # Remove commas from appraisal values
    appraisal_df['Appraised Value'] = appraisal_df['Appraised Value'].str.replace(',','').astype(int)

    # Create an array of the appraisal row addresses
    appraisal_addresses = list(appraisal_df['Property Address'])
    appraisal_addr_no = [address.split(' ')[0] for address in appraisal_addresses]
    appraisal_addr_str = [' '.join(address.split(' ')[1:-1]).replace('TX', '').strip() for address in appraisal_addresses]
    appraisal_zip_codes = [address.split(' ')[-1] for address in appraisal_addresses]
    appraisal_zip_codes = [zip.split('-')[0] for zip in appraisal_zip_codes]
    appraisal_values = list(appraisal_df['Appraised Value'])

    # Split these arrays into zipcode groups
    zip_bins = { appraisal_zip_codes[idx]: { 'addr_no': { }, 'addr_street': { }, 'appraisal': { } } for idx in range(len(appraisal_zip_codes)) }
    for idx in range(len(appraisal_zip_codes)):
        zip = appraisal_zip_codes[idx]
        zip_bins[zip]['addr_street'][idx] = appraisal_addr_str[idx]
        zip_bins[zip]['appraisal'][idx] = appraisal_values[idx]
        zip_bins[zip]['addr_no'][idx] = appraisal_addr_no[idx]

    # Initialize Appraisal column
    housing_df[year + ' Appraisal'] = 0

    # Log the data size
    print('Looking at ' + str(len(housing_df)) + ' houses and ' + str(len(appraisal_df)) + ' appraisal data points')

    # Parallelize computation
    count = 0
    match = 0
    with mp.Pool(int(mp.cpu_count() / 2)) as pool:
        results = pool.imap(findAppraisal, housing_df.iterrows(), chunksize=50)
        for result in results:
            count += 1
            if result['match']:
                housing_df.at[housing_df.index[result['idx']], year + ' Appraisal'] = result['val']
                match += 1
            print('\rProcessing house no: {}. Found {} matches so far.'.format(count, match), end="\r")

In [12]:
add_appraisal_data('2015')
add_appraisal_data('2019')



Building appraisal data for 2015
Looking at 61793 houses and 381875 appraisal data points
Processing house no: 61793. Found 39731 matches so far.

Building appraisal data for 2019
Looking at 61793 houses and 405597 appraisal data points
Processing house no: 61793. Found 40642 matches so far.

In [13]:
housing_df.to_csv('./data/raw_austin_housing_data_w_crime_density_poi_appraisals.csv')